# The problem

The problem of divide people in network to groups by their age group can be formulated in graph-theoretic term as:

We have a weighted graph $G=(V,E)$, whose edges $E$ corresponds to interactions and their weights $w(e), e\in E$, to the lenghts or times of interactions. 

In this context, we ask for partition $M=\{M_1, M_2,...,M_n\}$ of the set of vertices $V$ of the graph $G$ under the constraints:
1. $\forall v_j \in V, v_j \in M_i $ and $|V| = |M| = \sum_{i=1}^n M_i$ (Each person belongs to an age grop, and only one age group).
2. $w_i = \sum_{i=1}^n w(e)$ , where $e$ is "monochromatic edge" between nodes in $M_i$
3. $Y \in [0,1]$ is an assortativity bias (What part of the interactions is there between people of the same age group?)

Such that the error rate $W=|Y - \frac{1}{n}\sum_{i=1}^n w_i|$ is minimized

# Naïve approach

We will try all possible divisions and choose the most suitable one.

The number of ways to divide $n$ different objects into $r$ groups of sizes $a_1, a_2, a_3, …, a_r$ is equal to: \begin{equation} \frac{n!}{a_1! \cdot a_2! \cdot a_3!\cdot … \cdot a_r!} \end{equation}


In [1]:
import numpy as np

In [5]:
def naive_approach(matrix, age_groups_size, assortativity_bias =0.6):
    n = m.shape[0]
    all_possible_division = list(neclusters(range(n), len(age_groups_size)))
    best_div = []
    error_list = []
    best_error_rate = 1
    for div in all_possible_division:
        if [len(color) for color in div] == age_groups_size:
            cur_error = error_rate(matrix, assortativity_bias, div)
            error_list.append(cur_error)
            if cur_error < best_error_rate:
                best_error_rate = cur_error
                best_div = div
    best_error_rate = min(error_list) 
    print(f"Avarage error is {sum(error_list)/len(error_list)}, the best error is {best_error_rate}.")
    return best_div, best_error_rate

In [6]:
m = generate_random_adjacency_matrix(12)
print(m)
col_list = [3,4,5]
naive_approach(m, col_list, 0.7)

[[24 28  3 23 18 22 19  6  1 27 11 18]
 [28 26  8 23 29 19  8 16 19 24 13 28]
 [ 3  8 25  2 29 11 11  1 16 20 28 28]
 [23 23  2 13 12 21 26 11 13  5 15 18]
 [18 29 29 12 20  9 26 27 23 24 26 12]
 [22 19 11 21  9 26 14 28  8  9 29 24]
 [19  8 11 26 26 14  2 24 18 10  9  5]
 [ 6 16  1 11 27 28 24  3 23  7 21 22]
 [ 1 19 16 13 23  8 18 23 14 22  6 22]
 [27 24 20  5 24  9 10  7 22 23  5  9]
 [11 13 28 15 26 29  9 21  6  5 20 11]
 [18 28 28 18 12 24  5 22 22  9 11 15]]
Avarage error is 0.43725254196912955, the best error is 0.3516342573438146.


([[6, 7, 8], [2, 4, 9, 10], [0, 1, 3, 5, 11]], 0.3516342573438146)

#  Simulated annealing approach

We will start with random initial division and trying to optimize the error rate with tiny replacements in the groups.

In [21]:
import random 

def simulated_annealing_approach(matrix, age_groups_size, assortativity_bias =0.6, steps = 1, attempts = 10000):
    n = m.shape[0]
    indices = [i for i in range(n)]
    random.shuffle(indices)
    initial_division = []
    
    for group_size in age_groups_size:
        initial_division.append(indices[:group_size])
        indices = indices[group_size:]
    
    print(initial_division)
    cnt = 0
    best_error_rate = error_rate(matrix, assortativity_bias, initial_division)
    best_div = initial_division
    for i in range(attempts):
        temp_div = replace_in_list_of_lists(best_div)
        cur_error = error_rate(matrix, assortativity_bias, temp_div)
        
        if cur_error < best_error_rate:
            cnt+=1
            print(f"attempt: {i}, improvenent: {cnt}, best error rate {best_error_rate}")
            best_error_rate = cur_error
            best_div = temp_div
                
    return best_div, best_error_rate


In [23]:
m = generate_random_adjacency_matrix(50)
print(m)
col_list = [10]*5
simulated_annealing_approach(m, col_list, steps = 5, assortativity_bias = 0.6)

[[ 9 17  6 ... 26 26 15]
 [17  1 26 ...  7 23 16]
 [ 6 26 23 ... 12  8  1]
 ...
 [26  7 12 ... 28 11  1]
 [26 23  8 ... 11  8  4]
 [15 16  1 ...  1  4  5]]
[[4, 34, 11, 26, 7, 1, 8, 27, 19, 35], [46, 36, 6, 31, 18, 13, 14, 16, 17, 3], [20, 32, 37, 24, 5, 9, 40, 47, 49, 41], [29, 22, 42, 0, 15, 38, 10, 12, 21, 44], [48, 33, 23, 39, 45, 30, 25, 2, 28, 43]]
attempt: 0, improvenent: 1, best error rate 0.42772287979150825
attempt: 1, improvenent: 2, best error rate 0.4245194918014985
attempt: 2, improvenent: 3, best error rate 0.4235964817026821
attempt: 3, improvenent: 4, best error rate 0.42327071343251166
attempt: 18, improvenent: 5, best error rate 0.42061027255945266
attempt: 19, improvenent: 6, best error rate 0.41952437832555106
attempt: 21, improvenent: 7, best error rate 0.41832989466825926
attempt: 48, improvenent: 8, best error rate 0.41583233793028557
attempt: 54, improvenent: 9, best error rate 0.41572374850689536
attempt: 85, improvenent: 10, best error rate 0.4148550331197741

([[26, 44, 42, 4, 15, 8, 0, 2, 18, 49],
  [28, 33, 24, 21, 29, 10, 27, 13, 6, 30],
  [37, 20, 32, 36, 46, 22, 38, 31, 3, 45],
  [23, 1, 41, 39, 16, 7, 48, 11, 9, 19],
  [43, 12, 34, 35, 5, 17, 40, 25, 14, 47]],
 0.39818655662938424)

# Helpers

In [2]:
def generate_random_adjacency_matrix(n=10):
    b = np.random.randint(1,30,size=(n,n))
    b_symm = np.tril(b) + np.tril(b, -1).T
    return b_symm

In [3]:
import itertools
# create all clusters from l (list) elemnts into k groups
def neclusters(l, k):
    for labels in itertools.product(range(k), repeat=len(l)):
        partition = [[] for i in range(k)]
        for i, label in enumerate(labels):
            partition[label].append(l[i])
        yield partition
        

In [4]:
# node_colors is list of tuples
import numpy as np
from itertools import chain, combinations
def all_subsets(ss):
    return list(chain(*map(lambda x: combinations(ss, x), range(0, len(ss)+1))))

def sum_edeges_between_list(M, ls):
    comb = [i for i in all_subsets(ls) if len(i)==2]
    return sum([M[i[0]][i[1]] for i in comb])


def error_rate(M, Y, nodes_colors):
    total_weights = np.sum(M)/2
    sum_of_w_i = 0
    for color in nodes_colors:
        sum_of_w_i += sum_edeges_between_list(M, color)

    error_rate = abs(Y*total_weights-sum_of_w_i)/total_weights
    return error_rate

In [20]:
def replace_in_list_of_lists(l_of_l, steps = 1):
    for i in range(steps):
        num_lists = len(l_of_l)
        l1_inx, l2_inx = random.sample(range(num_lists), 2)
        len1 = len(l_of_l[l1_inx])
        len2 = len(l_of_l[l2_inx])
        elem1 = random.randint(0, len1-1)
        elem2 = random.randint(0, len2-1)

        temp = l_of_l[l1_inx][elem1]
        l_of_l[l1_inx][elem1] = l_of_l[l2_inx][elem2]
        l_of_l[l2_inx][elem2] = temp 
    return l_of_l